# Bulk RNA-Seq Analysis Training Demo

## Overview

This short tutorial demonstrates how to run an bulk RNA-Seq analysis workflow using a subset of reads from a zebrafish (*Danio rerio*) data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to estimate gene expression. This tutorial workflow uses a subset of sequence reads from a study published by Hartig et al (PubMed ID: [2746894](https://pubmed.ncbi.nlm.nih.gov/32746894/); Gene Expression Omnibus: [GSE137987](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137987)). In this study, the authors were interested in examining how chronic early life stress alters gene expression in adulthood. RNA samples were extracted from the blood of adult zebrafish who were exposed to cortisol or control conditions during just the first 5 days of development. There were three control (DMSO-treated) samples, and three cortisol-treated samples.

This tutorial only analyzes a subset of the read data (1 million reads per sample) from this experiment and aligns them to just chromosome 14 rather than the whole genome because analyzing the entire data set takes much longer to run. For those who are interested, we do provide another tutorial where the entire data set is analyzed but it takes several hours to run.

![RNA-Seq workflow](../../images/Tutorial_1_Workflow.png)

## STEP 1: Getting Started

<div class="alert alert-block alert-warning"> NOTE: This Jupyter Notebook was developed to run within a customized container on AWS with all software and tools pre-configured. If running without this customized container, you will need to install tools using the Miniforge environment setup instructions below before moving on to Step 2.</div>

### Without Container: Install Miniforge and Workflow Tools

Miniforge is a lightweight Conda distribution that offers a streamlined installation process and efficient package management. It provides access to a vast repository of packages.

Conda packages and environments are useful for several reasons. Conda packages contain metadata. This metadata includes information about what other programs the given software needs to be installed, in order to run. When installing a package with Conda, those other packages are automatically also installed. In this way, the user does not have to worry about manually installing each dependency. This makes installation quick and simple.

These packages are installed inside of environments, which are simply folders within the local installation of Conda. This has several benefits. Local installation means easier installation for non-admin users who may not have access to all system directories. Each environment can hold specific software with specific versions, and it easy to swap to different environments. In addition, the environments themselves are portable, as each environment contains a manifest on how to recreate that environment.

Miniforge itself is a Conda package manager, this means it requires Conda in order to work. It is used to install and update Conda packages, which it gets from a ‘channel’, or repository. It is an alternative to the native Conda package manager. It is often used for reasons of speed.

Bioconda is a ‘channel’, or repository, that the Mambaforge package manager can download packages from. It is a repository of Conda packages that are related to biology. These packages are versions of popular biology software that are curated and uploaded by contributing users.

The following code performs these steps:
- Downloads Miniforge or Mambaforge (you can use either based on preference)
- Installs Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
- Using miniforge and bioconda, installs the tools that will be used in this tutorial

<div class="alert alert-block alert-info">Tip: If using the Miniforge install, run the following code cells by removing the %%script false command. </div>

In [ ]:
%%script false --no-raise-error
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh > /dev/null

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge > /dev/null
!date +"%T"

Next, using miniforge and bioconda, install the tools that will be used in this tutorial.

In [ ]:
%%script false --no-raise-error
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin" 

# Now use mamba directly to install your software packages
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools pyathena samtools star rsem subread pigz -y > /dev/null

---------------------------------------
## If running from a container, as noted above, start with <b> STEP 2 </b> below:
## STEP 2: Define Threads & Setup Directory Structures

Create a set of directories to store the reads, reference sequence files, and output files.

In [ ]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data_14
!mkdir -p data_14/trimmed
!mkdir -p data_14/fastqc/
!mkdir -p data_14/fastqc_samples/
!mkdir -p data_14/raw_fastq/
!mkdir -p data_14/reference
!mkdir -p data_14/aligned_bam
!mkdir -p data_14/rsem_output
!mkdir -p data_14/reference/STAR_index_14

Specify the number of available threads based on the VM.

This is useful for later tools such as trimmomatic, or STAR.

In [ ]:
import multiprocessing
import os

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

## STEP 3: Downloading relevant files



### STEP 3.1: Finding run accession numbers.


This code retrieves the SRR IDs associated with project PRJNA892075 from the NCBI database and saves them to a .txt file

In [ ]:
!esearch -db sra -query "PRJNA318296" | efetch -format runinfo | cut -d',' -f1 | tail -n +2 > accs.txt
!cat accs.txt

### STEP 3.2: Download subsampling FASTQ files from S3 bucket

In order for this tutorial to run quickly, we will only analyze 1,000,000 reads from each of the eight samples in PRJNA892075. These subsampled files have been posted on a publicly accessible S3 bucket.

The code will connect to the S3 bucket and download the paired-end FASTQ files for each SRR.

In [ ]:
#Load SRR IDs from accs.txt (if available)
with open('accs.txt', 'r') as f:
    accs = [line.strip() for line in f.readlines()]
    
# Define the path to S3 bucket

s3_path = 's3://nigms-sandbox/bulk-scRNAseq/fastqfiles/'

In [ ]:
for acc in accs:
    !wget https://sra-data-athena.s3.amazonaws.com/fastqfiles/subsampled_chr14_{acc}_1.fastq -P data_14/raw_fastq/
    !wget https://sra-data-athena.s3.amazonaws.com/fastqfiles/subsampled_chr14_{acc}_2.fastq -P data_14/raw_fastq/

### STEP 3.3: Download reference genome and annotation files that will be used by STAR and RSEM

This step downloads and unzips reference files for the zebrafish genome and annotations needed by STAR. These reference files will be used by STAR to align RNA-seq reads during the analysis. To expedite this tutorial, we'll filter the zebrafish genome and annotation to include only data from chromosome 19.

In [ ]:
! wget ftp://ftp.ensembl.org/pub/release-104/fasta/danio_rerio/dna/Danio_rerio.GRCz11.dna.primary_assembly.fa.gz -O data_14/reference/zebrafish_genome.fa.gz
! wget ftp://ftp.ensembl.org/pub/release-104/gtf/danio_rerio/Danio_rerio.GRCz11.104.gtf.gz -O data_14/reference/zebrafish_annotation.gtf.gz
! wget -O data_14/reference/zebrafish_feature_table.txt.gz https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/035/GCF_000002035.6_GRCz11/GCF_000002035.6_GRCz11_feature_table.txt.gz

In [ ]:
!gunzip -f data_14/reference/zebrafish_genome.fa.gz 
!gunzip -f data_14/reference/zebrafish_annotation.gtf.gz
!gunzip -f data_14/reference/zebrafish_feature_table.txt.gz

In [ ]:
# Use samtools to extract only chr14 from the reference genome
!samtools faidx data_14/reference/zebrafish_genome.fa 14 > data_14/reference/zebrafish_genome_chr14.fa 

# Subset the GTF file to include only chr14 annotations
!grep "^14" data_14/reference/zebrafish_annotation.gtf > data_14/reference/zebrafish_annotation_chr14.gtf 

### STEP 3.4: Copy data file for Trimmomatic


One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [ ]:
!wget -P data_14/trimmed/ https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/TruSeq3-PE.fa

### STEP 4: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Trimmomatic takes an input, in this case a forward and reverse FASTQ file, and the user names the outputs. In this case the first two outputs are the ‘paired’ and ‘orphaned’ trimmed files for the forward reads. The second two are the ‘paired’ and ‘orphaned’ trimmed files for the reverse reads.

Paired in this case means the forward and reverse sequences were able to be aligned to each other after trimming. Orphaned, or unpaired, typically means one of the reads was discarded as a result of trimming, and only the forward or reverse read survived.

Here we take only the file with paired reads, as there are only a few orphaned reads, and including orphaned reads can complicate different downstream analyses. Unless there is a significant amount of them, or a specific reason to use them, it is generally easier to discard unpaired reads. Also in the interest of simplicity and speed, we proceed in further steps with just using the paired-end reads of just the forward-end read files, which in this context is sufficient, however in different contexts, using both forward and reverse is often preferable.

The last part of the command specifies how the trimming is done: `ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36`

<u>In greater detail</u>:

`ILLUMINACLIP:TruSeq3-PE.fa` refers to which adapters should be cut from the reads.

`2:30:10:2` refers to various metrics, which are recommended defaults.

`2` refers to the seed mismatch. This refers to the amount of mismatches a 'seed' may have in aligning to a possible adapter.

`30` refers to the palindrome clip threshold. This refers to the similarity score. If forward and reverse reads, after having an adapter attached to them, are greater than this score, trimming of adapter fragments will be performed. Forward reads are clipped, and reverse reads dropped.

`10` refers to the simple clip threshold. The basic, and alternative method to palindromic searching for adapters. Adapters are tested against reads and if sufficiently matched (above the threshold, in this case, 10), they are clipped.

`2` refers to the minimum adapter fragment length in palindrome mode.

`LEADING:3` refers to trimming bases from the start of a read. Bases at the start of a read will continue to be trimmed, sequentially, as long as the bases remain below a PHRED score of 3.

`TRAILING:3` refers to the same as above, but at the end of a read.

`MINLEN:36`, the read is discarded if below this length.

Greater information about parameters can be found in the trimmomatic documentation.

In [ ]:
!cat accs.txt | xargs -I {} \
trimmomatic PE -threads $THREADS \
'data_14/raw_fastq/subsampled_chr14_{}_1.fastq' 'data_14/raw_fastq/subsampled_chr14_{}_2.fastq' \
'data_14/trimmed/{}_1_trimmed.fastq' 'data_14/trimmed/{}_1_trimmed_unpaired.fastq' \
'data_14/trimmed/{}_2_trimmed.fastq' 'data_14/trimmed/{}_2_trimmed_unpaired.fastq' \
ILLUMINACLIP:data_14/trimmed/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

### STEP 5: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

Because jupyter is at its core a python editor, we can use python code and html support to display results in-line.

FastQC looks for different characteristics of quality in reads. It is very rare that every metric will pass. In many cases, they serve as warnings, which should be compared to the context of the experiment. For instance, here, per base sequence content, sequence length distribution, sequence duplication levels, and overrepresented sequences all throw warnings. Per base sequence content routinely fails in RNA-sequencing in the first 15~ or so bases due to biased fragmentation. In most of our samples, this is where we see the failure (20% or more difference between A/T or G/C), and so is not unexpected. The overrepresented sequences can be BLASTed to show the majority of them are ribosomal RNA. Ribosomal RNA contamination is also common and will not be indexed later, and so not a large concern. Other metrics look good.

In [ ]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data_14/trimmed/{}_1_trimmed.fastq data_14/trimmed/{}_2_trimmed.fastq -o data_14/fastqc_samples/

In [ ]:
from IPython.display import IFrame
IFrame(src='./data_14/fastqc_samples/SRR3371417_1_trimmed_fastqc.html', width=800, height=600)

### STEP 6: MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

Being able to use python with bash also means we can seamlessly use popular python packages, such as pandas, to interact with or view the files we create.

In [ ]:
#!multiqc -f data_14/fastqc_samples/
!multiqc -f -o data_14/multiqc_samples/ data_14/fastqc_samples/

In [ ]:
!pip install jupyterquiz

In [ ]:
import json
from jupyterquiz import display_quiz

#display Multiple Choice quiz
display_quiz("../../quiz/bulkRNADR1.json")

### STEP 7: Preparing the STAR-Compatible RSEM Reference

The provided code snippet demonstrates the process of preparing a reference genome and annotation files for RNA-Seq analysis using RSEM and STAR. The initial command, rsem-prepare-reference, creates a reference index for the zebrafish genome based on the provided GTF annotation file (zebrafish_annotation_chr14.gtf). This index, along with the reference genome sequence (zebrafish_genome_chr14.fa), is essential for quantifying gene and isoform expression using RSEM. The --star option ensures compatibility with STAR, a popular transcriptome aligner.

However, the subsequent STAR genome generation step encounters a warning due to an excessively large --genomeSAindexNbases parameter. This parameter controls the size of the genome's suffix array index, which is used for efficient alignment. If the parameter is too large for the given genome size, it can lead to segmentation faults during the mapping process. To address this issue, the code adjusts the --genomeSAindexNbases parameter to 11, which is more appropriate for the zebrafish genome. This modification ensures that the STAR genome generation process completes successfully without errors.

In [ ]:
# Preparing the reference genome
!rsem-prepare-reference --gtf data_14/reference/zebrafish_annotation_chr14.gtf --star -p $THREADS data_14/reference/zebrafish_genome_chr14.fa zebrafish_reference > /dev/null

In [ ]:
!STAR  --runThreadN $THREADS  --runMode genomeGenerate  --genomeDir .  --genomeFastaFiles data_14/reference/zebrafish_genome_chr14.fa  --sjdbGTFfile data_14/reference/zebrafish_annotation_chr14.gtf  --sjdbOverhang 100  --outFileNamePrefix zebrafish_reference  --genomeSAindexNbases 11 > /dev/null

### STEP 8: Automated RNA-seq Quantification with RSEM

This code automates the process of quantifying gene expression from RNA-seq data using RSEM. It reads a list of SRR accession numbers from a text file, iterates over each accession, and executes the RSEM command to calculate gene expression levels. The code assumes that the trimmed FASTQ files for each SRR are located in a specified directory (data_14/trimmed/) and uses a previously generated reference index (zebrafish_reference). The output files are saved in the data_14/rsem_output directory, with filenames based on the corresponding SRR accession number. This script simplifies the workflow for analyzing multiple RNA-seq samples using RSEM.

In [ ]:
import os

# Ensure you've set the path to the RSEM binary
# Read the SRR accessions from the file
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]

# Define the output directory
output_dir = "data_14/rsem_output"

# Loop through each SRR accession and run rsem-calculate-expression
for srr in srr_accessions:
    !rsem-calculate-expression -p $THREADS --paired-end --star \
    data_14/trimmed/{srr}_1_trimmed.fastq data_14/trimmed/{srr}_2_trimmed.fastq zebrafish_reference data_14/rsem_output/{srr} > /dev/null

### STEP 9: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data_14/rsem_output'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Sort the DataFrame by TPM values in descending order and get the top 10 genes
    top_10_genes = df.sort_values(by='TPM', ascending=False).head(10)

    # Print the top 10 genes with their TPM values
    print(f"Top 10 Genes by TPM for {srr_id}:")
    print(top_10_genes[['gene_id', 'TPM']])

In [ ]:
#display Multiple Choice quiz
display_quiz("../../quiz/bulkRNADR2.json")

Top 10 most highly expressed genes in the double lysogen samples.


### STEP 10: Report the expression of irg1l for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
- `Name`
- `Length`
- `EffectiveLength`
- `TPM`
- `NumReads`

In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data_14/rsem_output'

# Target gene ID
target_gene = 'ENSDARG00000092553'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Filter for the target gene
    target_gene_data = df[df['gene_id'] == target_gene]

    # Print the target gene's TPM value for the SRR ID
    print(f"TPM for {target_gene} in {srr_id}: {target_gene_data['TPM'].values[0]}")

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One (Extended):](Tutorial_1B_Extended_zebrafish.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset.

[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_zebrafish.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.
